# Deep Gaussian Processes with Doubly Stochastic VI

In this notebook, we provide a GPyTorch implementation of deep Gaussian processes, where training and inference is performed using the method of Salimbeni et al., 2017 (https://arxiv.org/abs/1705.08933) adapted to CG-based inference.

We'll be training a simple two layer deep GP on the `elevators` UCI dataset.

In [1]:
%set_env CUDA_VISIBLE_DEVICES=0

import torch
import gpytorch
from torch.nn import Linear
from gpytorch.means import ConstantMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import AbstractVariationalGP, GP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood


env: CUDA_VISIBLE_DEVICES=0


In [2]:
from gpytorch.models.deep_gps import AbstractDeepGPLayer, AbstractDeepGP, DeepLikelihood 

## Loading Data

For this example notebook, we'll be using the `elevators` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a ~400 KB dataset file to the current directory.

In [3]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor
import numpy as np

if not os.path.isfile('elevators.mat'):
    print('Downloading \'elevators\' UCI dataset...')
    urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk', 'elevators.mat')
    
data = torch.Tensor(loadmat('elevators.mat')['data'])
X = data[:, :-1]
y = data[:, -1]

N = data.shape[0]
np.random.seed(0)
data = data[np.random.permutation(np.arange(N)),:]

train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

mean = train_x.mean(dim=-2, keepdim=True)
std = train_x.std(dim=-2, keepdim=True) + 1e-6
train_x = (train_x - mean) / std
test_x = (test_x - mean) / std

mean,std = train_y.mean(),train_y.std()
train_y = (train_y - mean) / std
test_y = (test_y - mean) / std

In [4]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

# Defining GP layers

In GPyTorch, defining a GP involves extending one of our abstract GP models and defining a `forward` method that returns the prior. For deep GPs, things are similar, but there are two abstract GP models that must be overwritten: one for hidden layers and one for the deep GP model itself.

In the next cell, we define an example deep GP hidden layer. This looks very similar to every other variational GP you might define. However, there are a few key differences:

1. Instead of extending `AbstractVariationalGP`, we extend `AbstractDeepGPLayer`.
2. `AbstractDeepGPLayers` need a number of input dimensions, a number of output dimensions, and a number of samples. This is kind of like a linear layer in a standard neural network -- `input_dims` defines how many inputs this hidden layer will expect, and `output_dims` defines how many hidden GPs to create outputs for.

In [5]:
class ToyDeepGPHiddenLayer(AbstractDeepGPLayer):
    def __init__(self, input_dims, output_dims, num_inducing=512):
        if output_dims is None:
            inducing_points = torch.randn(num_inducing, input_dims)
        else:
            inducing_points = torch.randn(output_dims, num_inducing, input_dims)

        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=num_inducing,
            batch_shape=torch.Size([output_dims]) if output_dims is not None else torch.Size([])
        )

        variational_strategy = VariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        )

        super(ToyDeepGPHiddenLayer, self).__init__(variational_strategy, input_dims, output_dims)

        self.mean_module = ConstantMean(batch_size=output_dims)
        self.covar_module = ScaleKernel(
            RBFKernel(batch_size=output_dims, ard_num_dims=input_dims),
            batch_size=output_dims, ard_num_dims=None
        )
        
        self.linear_layer = Linear(input_dims, 1)

    def forward(self, x):
        mean_x = self.mean_module(x) # self.linear_layer(x).squeeze(-1)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)
    
    def __call__(self, x, *other_inputs, **kwargs):
        """
        Overriding __call__ isn't strictly necessary, but it lets us add concatenation based skip connections
        easily. For example, hidden_layer2(hidden_layer1_outputs, inputs) will pass the concatenation of the first
        hidden layer's outputs and the input data to hidden_layer2.
        """
        if len(other_inputs):
            if isinstance(x, gpytorch.distributions.MultitaskMultivariateNormal):
                x = x.rsample()

            processed_inputs = [
                inp.unsqueeze(0).expand(self.num_samples, *inp.shape)
                for inp in other_inputs
            ]

            x = torch.cat([x] + processed_inputs, dim=-1)

        return super().__call__(x, are_samples=bool(len(other_inputs)))
    

# Building the model

Now that we've defined a class for our hidden layers and a class for our output layer, we can build our deep GP. To do this, we create a `Module` whose forward is simply responsible for forwarding through the various layers.

This also allows for various network connectivities easily. For example calling,
```
hidden_rep2 = self.second_hidden_layer(hidden_rep1, inputs)
```
in forward would cause the second hidden layer to use both the output of the first hidden layer and the input data as inputs, concatenating the two together.

In [6]:
class DeepGP(AbstractDeepGP):
    def __init__(self, train_x_shape):
        hidden_layer = ToyDeepGPHiddenLayer(
            input_dims=train_x_shape[-1],
            output_dims=6
        )
        
        last_layer = ToyDeepGPHiddenLayer(
            input_dims=hidden_layer.output_dims,
            output_dims=None,
        )
        
        super().__init__(last_layer)
        
        self.hidden_layer = hidden_layer
        self.last_layer = last_layer
        self.likelihood = DeepLikelihood(GaussianLikelihood())
    
    def forward(self, inputs):
        hidden_rep1 = self.hidden_layer(inputs)
        output = self.last_layer(hidden_rep1)
        return output
    
    def predict(self, x):
        with gpytorch.settings.fast_computations(log_prob=False, solves=False), torch.no_grad():
            preds = self(x)
        predictive_means = preds.mean
        predictive_variances = preds.variance
        
        return predictive_means, predictive_variances

In [7]:
model = DeepGP(train_x.shape).cuda()

# Likelihood

Because deep GPs use some amounts of internal sampling (even in the stochastic variational setting), we need to handle the likelihood in a slightly different way. In the future, we anticipate `DeepLikelihood` being a general wrapper around an arbitrary likelihood once likelihoods become a little more general purpose, but for now we simply define a `DeepGaussianLikelihood` to use for regression.

# Training the model

The training loop for a deep GP looks similar to a standard GP model with stochastic variational inference, but there are a few differences:

1. Because the output of a deep GP is actually num_outputs x num_samples Gaussians rather than a single Gaussian, we need to expand the labels to be num_outputs x num_samples x minibatch_size before calling the ELBO.
2. Because deep GPs involve a few added loss terms and normalize slightly differently, we created the `VariationalELBO` above with `combine_terms=False`. This just lets us do the extra normalization we need to make the math work out.

In [8]:
num_epochs = 60

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
], lr=0.01)
mll = VariationalELBO(model.likelihood, model, train_x.shape[-2])

import time

with gpytorch.settings.fast_computations(log_prob=False, solves=False):
    for i in range(num_epochs):
        for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):
            start_time = time.time()
            optimizer.zero_grad()
            
            output = model(x_batch)
            loss = -mll(output, y_batch)
            print('Epoch %d [%d/%d] - Loss: %.3f - - Time: %.3f' % (i + 1, minibatch_i, len(train_loader), loss.item(), time.time() - start_time))

            loss.backward()
            optimizer.step()

Epoch 1 [0/13] - Loss: 1.911 - - Time: 0.519
Epoch 1 [1/13] - Loss: 1.919 - - Time: 0.158
Epoch 1 [2/13] - Loss: 1.981 - - Time: 0.154
Epoch 1 [3/13] - Loss: 1.917 - - Time: 0.154
Epoch 1 [4/13] - Loss: 1.870 - - Time: 0.153
Epoch 1 [5/13] - Loss: 1.850 - - Time: 0.154
Epoch 1 [6/13] - Loss: 1.852 - - Time: 0.154
Epoch 1 [7/13] - Loss: 1.828 - - Time: 0.154
Epoch 1 [8/13] - Loss: 1.800 - - Time: 0.151
Epoch 1 [9/13] - Loss: 1.778 - - Time: 0.154
Epoch 1 [10/13] - Loss: 1.805 - - Time: 0.153
Epoch 1 [11/13] - Loss: 1.738 - - Time: 0.154
Epoch 1 [12/13] - Loss: 1.753 - - Time: 0.151
Epoch 2 [0/13] - Loss: 1.753 - - Time: 0.153
Epoch 2 [1/13] - Loss: 1.745 - - Time: 0.153
Epoch 2 [2/13] - Loss: 1.702 - - Time: 0.153
Epoch 2 [3/13] - Loss: 1.707 - - Time: 0.153
Epoch 2 [4/13] - Loss: 1.697 - - Time: 0.153
Epoch 2 [5/13] - Loss: 1.712 - - Time: 0.153
Epoch 2 [6/13] - Loss: 1.737 - - Time: 0.154
Epoch 2 [7/13] - Loss: 1.669 - - Time: 0.154
Epoch 2 [8/13] - Loss: 1.666 - - Time: 0.154
Epoch 2

Epoch 14 [11/13] - Loss: 1.418 - - Time: 0.155
Epoch 14 [12/13] - Loss: 1.444 - - Time: 0.153
Epoch 15 [0/13] - Loss: 1.410 - - Time: 0.155
Epoch 15 [1/13] - Loss: 1.515 - - Time: 0.136
Epoch 15 [2/13] - Loss: 1.505 - - Time: 0.156
Epoch 15 [3/13] - Loss: 1.403 - - Time: 0.155
Epoch 15 [4/13] - Loss: 1.530 - - Time: 0.155
Epoch 15 [5/13] - Loss: 1.463 - - Time: 0.155
Epoch 15 [6/13] - Loss: 1.435 - - Time: 0.155
Epoch 15 [7/13] - Loss: 1.401 - - Time: 0.155
Epoch 15 [8/13] - Loss: 1.467 - - Time: 0.155
Epoch 15 [9/13] - Loss: 1.395 - - Time: 0.155
Epoch 15 [10/13] - Loss: 1.428 - - Time: 0.155
Epoch 15 [11/13] - Loss: 1.399 - - Time: 0.156
Epoch 15 [12/13] - Loss: 1.447 - - Time: 0.153
Epoch 16 [0/13] - Loss: 1.422 - - Time: 0.155
Epoch 16 [1/13] - Loss: 1.449 - - Time: 0.156
Epoch 16 [2/13] - Loss: 1.441 - - Time: 0.156
Epoch 16 [3/13] - Loss: 1.423 - - Time: 0.156
Epoch 16 [4/13] - Loss: 1.450 - - Time: 0.156
Epoch 16 [5/13] - Loss: 1.456 - - Time: 0.156
Epoch 16 [6/13] - Loss: 1.415

Epoch 28 [7/13] - Loss: 1.471 - - Time: 0.155
Epoch 28 [8/13] - Loss: 1.450 - - Time: 0.155
Epoch 28 [9/13] - Loss: 1.449 - - Time: 0.156
Epoch 28 [10/13] - Loss: 1.459 - - Time: 0.156
Epoch 28 [11/13] - Loss: 1.430 - - Time: 0.156
Epoch 28 [12/13] - Loss: 1.383 - - Time: 0.153
Epoch 29 [0/13] - Loss: 1.424 - - Time: 0.152
Epoch 29 [1/13] - Loss: 1.421 - - Time: 0.155
Epoch 29 [2/13] - Loss: 1.482 - - Time: 0.155
Epoch 29 [3/13] - Loss: 1.427 - - Time: 0.155
Epoch 29 [4/13] - Loss: 1.399 - - Time: 0.136
Epoch 29 [5/13] - Loss: 1.416 - - Time: 0.156
Epoch 29 [6/13] - Loss: 1.445 - - Time: 0.156
Epoch 29 [7/13] - Loss: 1.460 - - Time: 0.155
Epoch 29 [8/13] - Loss: 1.495 - - Time: 0.155
Epoch 29 [9/13] - Loss: 1.398 - - Time: 0.156
Epoch 29 [10/13] - Loss: 1.396 - - Time: 0.156
Epoch 29 [11/13] - Loss: 1.449 - - Time: 0.156
Epoch 29 [12/13] - Loss: 1.419 - - Time: 0.153
Epoch 30 [0/13] - Loss: 1.457 - - Time: 0.155
Epoch 30 [1/13] - Loss: 1.432 - - Time: 0.155
Epoch 30 [2/13] - Loss: 1.44

Epoch 42 [3/13] - Loss: 0.726 - - Time: 0.155
Epoch 42 [4/13] - Loss: 0.739 - - Time: 0.156
Epoch 42 [5/13] - Loss: 0.746 - - Time: 0.155
Epoch 42 [6/13] - Loss: 0.723 - - Time: 0.155
Epoch 42 [7/13] - Loss: 0.721 - - Time: 0.155
Epoch 42 [8/13] - Loss: 0.693 - - Time: 0.156
Epoch 42 [9/13] - Loss: 0.715 - - Time: 0.155
Epoch 42 [10/13] - Loss: 0.717 - - Time: 0.155
Epoch 42 [11/13] - Loss: 0.706 - - Time: 0.156
Epoch 42 [12/13] - Loss: 0.726 - - Time: 0.153
Epoch 43 [0/13] - Loss: 0.717 - - Time: 0.155
Epoch 43 [1/13] - Loss: 0.718 - - Time: 0.155
Epoch 43 [2/13] - Loss: 0.717 - - Time: 0.154
Epoch 43 [3/13] - Loss: 0.718 - - Time: 0.155
Epoch 43 [4/13] - Loss: 0.674 - - Time: 0.155
Epoch 43 [5/13] - Loss: 0.675 - - Time: 0.155
Epoch 43 [6/13] - Loss: 0.693 - - Time: 0.155
Epoch 43 [7/13] - Loss: 0.684 - - Time: 0.136
Epoch 43 [8/13] - Loss: 0.680 - - Time: 0.156
Epoch 43 [9/13] - Loss: 0.698 - - Time: 0.156
Epoch 43 [10/13] - Loss: 0.655 - - Time: 0.156
Epoch 43 [11/13] - Loss: 0.661

Epoch 55 [12/13] - Loss: 0.567 - - Time: 0.152
Epoch 56 [0/13] - Loss: 0.568 - - Time: 0.155
Epoch 56 [1/13] - Loss: 0.599 - - Time: 0.156
Epoch 56 [2/13] - Loss: 0.537 - - Time: 0.155
Epoch 56 [3/13] - Loss: 0.520 - - Time: 0.155
Epoch 56 [4/13] - Loss: 0.557 - - Time: 0.156
Epoch 56 [5/13] - Loss: 0.583 - - Time: 0.156
Epoch 56 [6/13] - Loss: 0.535 - - Time: 0.156
Epoch 56 [7/13] - Loss: 0.582 - - Time: 0.155
Epoch 56 [8/13] - Loss: 0.585 - - Time: 0.155
Epoch 56 [9/13] - Loss: 0.562 - - Time: 0.156
Epoch 56 [10/13] - Loss: 0.534 - - Time: 0.155
Epoch 56 [11/13] - Loss: 0.584 - - Time: 0.155
Epoch 56 [12/13] - Loss: 0.553 - - Time: 0.153
Epoch 57 [0/13] - Loss: 0.580 - - Time: 0.154
Epoch 57 [1/13] - Loss: 0.572 - - Time: 0.155
Epoch 57 [2/13] - Loss: 0.558 - - Time: 0.155
Epoch 57 [3/13] - Loss: 0.534 - - Time: 0.155
Epoch 57 [4/13] - Loss: 0.575 - - Time: 0.155
Epoch 57 [5/13] - Loss: 0.550 - - Time: 0.155
Epoch 57 [6/13] - Loss: 0.548 - - Time: 0.156
Epoch 57 [7/13] - Loss: 0.538 

# Make predictions and get an RMSE

The output distribution of a deep GP in this framework is actually a mixture of `num_samples` Gaussians for each output. We get predictions the same way with all GPyTorch models, but we do currently need to do some reshaping to get the means and variances in a reasonable form.

SVGP gets an RMSE of around 0.41 after 60 epochs of training, so overall getting an RMSE of 0.34 out of a 2 layer deep GP without much tuning involved is pretty good!

In [10]:
import gpytorch
model.eval()
predictive_means, predictive_variances = model.predict(test_x)

rmse = torch.mean(torch.pow(predictive_means.mean(0) - test_y, 2)).sqrt()
print(rmse)

tensor(0.3723, device='cuda:0')
